# マルコフモデル

*Markov Model*

マルコフ過程に従う確率モデル。マルコフ連鎖とも。  
マルコフ過程とは、ある状態から次の状態へ遷移する確率が、その状態のみに依存する確率過程のこと。

In [1]:
import random


---

## 文章生成

マルコフモデルを用いて文章を生成する。状態を単語とし、ある単語の次に続く単語の確率を予測する。

学習データとして、いくつかの単語列（=文章）を用意する。学習データに出現する全ての単語を対象に、その次に続く単語とその確率を記録することで、マルコフモデルが完成する。確率は単語の出現頻度から定める。

例として以下の単語列を考えてみよう。

- 今日 は いい 天気 です
- 私 は 今日 カレー を 食べ ました
- 私 は カレー が 好き です

この3つの単語列を学習させてマルコフモデルを作成する。

まず、1つ目に選ばれる可能性のある単語として「今日」と「私」がある。どちらが選ばれるかはランダムであるが、「今日」は1回、「私」は2回出現しているので、「今日」が選ばれる確率は1/3、「私」が選ばれる確率は2/3と定義する。  
次に、この次に選ばれる可能性のある単語と確率を定義する。「今日」の次に来る可能性がある単語は「は」と「カレー」で、出現頻度は同じであるため確率は一様となる。  
この流れで全ての単語について次に続く単語と確率を定義するとこうなる。 

In [2]:
data = [
    "今日 は いい 天気 です",
    "私 は 今日 カレー を 食べ ました",
    "私 は カレー が 好き です"
]

words = {}
for sent in data:
    sent = sent.split()
    for w1, w2 in zip(sent[:-1], sent[1:]):
        if w1 not in words:
            words[w1] = {}
        if w2 not in words[w1]:
            words[w1][w2] = 0
        words[w1][w2] += 1

for w1 in words:
    total = sum(words[w1].values())
    for w2 in words[w1]:
        words[w1][w2] /= total

words

{'今日': {'は': 0.5, 'カレー': 0.5},
 'は': {'いい': 0.3333333333333333,
  '今日': 0.3333333333333333,
  'カレー': 0.3333333333333333},
 'いい': {'天気': 1.0},
 '天気': {'です': 1.0},
 '私': {'は': 1.0},
 'カレー': {'を': 0.5, 'が': 0.5},
 'を': {'食べ': 1.0},
 '食べ': {'ました': 1.0},
 'が': {'好き': 1.0},
 '好き': {'です': 1.0}}

「今日」の次は「は」と「カレー」が$\frac{1}{2}$ずつ、「は」の次は「いい」「今日」「カレー」が$\frac{1}{3}$ずつという風に確率を定義することが出来た。

では、この確率を用いて文章を生成してみる。

In [3]:
def generate(words, start_words):
    word, = random.choices(list(start_words), weights=start_words.values())
    sentence = [word]
    while word in words:
        next_words = words[word]
        word, = random.choices(list(next_words), weights=next_words.values())
        sentence.append(word)
    return ' '.join(sentence)

for _ in range(10):
    sentence = generate(words, {"今日": 1/3, "私": 2/3})
    print(sentence)

私 は 今日 カレー を 食べ ました
今日 カレー が 好き です
今日 は いい 天気 です
今日 カレー が 好き です
私 は いい 天気 です
私 は 今日 カレー を 食べ ました
今日 は 今日 は 今日 カレー を 食べ ました
私 は いい 天気 です
今日 カレー が 好き です
私 は いい 天気 です


自然な文章もあれば、意味不明な文章もあるだろう。単語を予測する際に直前の単語のみを参照しているため、自然な文章を生成することは難しい。

### markovify

マルコフモデルを用いて文章を生成するためのライブラリ。

- [jsvine/markovify: A simple, extensible Markov chain generator.](https://github.com/jsvine/markovify)

これを使うと簡単にマルコフモデルを用いた言語モデルを実装できる。

In [4]:
import markovify

In [5]:
data = [
    "今日 は いい 天気 です",
    "私 は 今日 カレー を 食べ ました",
    "私 は カレー が 好き です"
]
model = markovify.Text(data, state_size=1)

これで学習が完了した。文章を生成してみる。

In [6]:
for _ in range(10):
    sentence = model.make_sentence()
    print(sentence)

私 は カレー を 食べ ました
私 は カレー を 食べ ました
私 は カレー を 食べ ました
私 は いい 天気 です
今日 カレー が 好き です
私 は カレー を 食べ ました
私 は カレー を 食べ ました
私 は 今日 カレー が 好き です
私 は いい 天気 です
今日 カレー が 好き です



---

## 一般的なデータセットの活用

ここまで、私が用意した3つの短文を学習データとして言語モデルを作成した。本節ではもう少し大きなデータセットを用いて言語モデルを作成する。

### wiki40b

Wikipediaの記事を集めたデータセット。  
Tensorflow Datasetsに収録されているものを使用する。

[wiki40b  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wiki40b?hl=en#wiki40bja)

In [7]:
import tensorflow_datasets as tfds
ds = tfds.load('wiki40b/ja', split='test')
ds = list(ds.as_numpy_iterator())

2023-06-18 23:52:59.472848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 23:52:59.943937: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-06-18 23:53:00.444905: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-18 23:53:00.465202: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the requir

以下の様な辞書がまとまっている

In [8]:
ex = ds[0]
ex

{'text': b'\n_START_ARTICLE_\n\xe3\x83\x93\xe3\x83\xbc\xe3\x83\x88\xe3\x81\x9f\xe3\x81\x91\xe3\x81\x97\xe3\x81\xae\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\n_START_SECTION_\n\xe6\xa6\x82\xe8\xa6\x81\n_START_PARAGRAPH_\n\xe3\x80\x8c\xe6\x95\x99\xe7\xa7\x91\xe6\x9b\xb8\xe3\x81\xab\xe3\x81\xaf\xe6\xb1\xba\xe3\x81\x97\xe3\x81\xa6\xe8\xbc\x89\xe3\x82\x89\xe3\x81\xaa\xe3\x81\x84\xe3\x80\x8d\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xae\xe8\xac\x8e\xe3\x82\x84\xe3\x81\x97\xe3\x81\x8d\xe3\x81\x9f\xe3\x82\x8a\xe3\x82\x92\xe5\xa4\x9a\xe8\xa7\x92\xe7\x9a\x84\xe3\x81\xab\xe6\xa4\x9c\xe8\xa8\xbc\xe3\x81\x97\xe3\x80\x81\xe6\x97\xa5\xe6\x9c\xac\xe4\xba\xba\xe3\x81\xaeDNA\xe3\x82\x92\xe8\xa7\xa3\xe6\x98\x8e\xe3\x81\x99\xe3\x82\x8b\xe3\x80\x82_NEWLINE_\xe6\x96\xb0\xe6\x98\xa5\xe7\x95\xaa\xe7\xb5\x84\xe3\x81\xa8\xe3\x81\x97\xe3\x81\xa6\xe5\xae\x9a\xe6\x9c\x9f\xe7\x9a\x84\xe3\x81\xab\xe6\x

テキストの中身はこんな感じ

In [9]:
print(ex['text'].decode())


_START_ARTICLE_
ビートたけしの教科書に載らない日本人の謎
_START_SECTION_
概要
_START_PARAGRAPH_
「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。_NEWLINE_新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。


セクションごとにまとめる

In [10]:
data = []
for sample in ds:
    text = sample['text'].decode()
    sections = text.split('_START_SECTION_')
    for section in sections[1:]:
        sentence = section.split('_START_PARAGRAPH_')[1]
        sentence = sentence.replace('_NEWLINE_', '')
        sentence = sentence.replace('\n', '')
        data.append(sentence)

print('num of data:', len(data))
data[:5] # examples

num of data: 89698


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。',
 '香川県内の農業協同組合の信用事業を統括する県域農協系金融機関であり、県内農業協同組合を会員とする。香川県は全県単一農協の香川県農業協同組合となったが、先に単一農協となった奈良県や沖縄県のケースと異なり、信連の統合は行われなかった。通称は「JA香川信連」または「JAバンク香川」。統一金融機関コードは3037。主に法人顧客を中心としており、個人取引は殆どない。県内の大型商業施設にある、他金融機関管理の共同ATMには香川信連の管轄のものがある。',
 '534年（永熙3年）、独孤信の子として生まれた。独孤信が父母妻子を捨てて長安に入ったため、独孤羅は東魏に取り残されて高氏の虜囚となった。独孤

### 分かち書き

テキストを単語ごとに分割する。英語は元から単語ごとに分割されているが、日本語はそうではないので別途行う必要がある。  
分かち書きには形態素解析器を使用する。色々な種類があるが、ここでは**MeCab**を使用する。

In [11]:
import MeCab

In [12]:
tagger = MeCab.Tagger('-Owakati') # 出力形式を分かち書きに指定
result = tagger.parse('私は猫が好きです。')
result

'私 は 猫 が 好き です 。 \n'

これを使って、学習データの全てを単語ごとに分割する。

In [13]:
data_wakati = []
for sentence in data:
    data_wakati.append(tagger.parse(sentence).strip())

data_wakati[:5] # examples

['「 教科 書 に は 決して 載ら ない 」 日本 人 の 謎 や しきたり を 多角 的 に 検証 し 、 日本 人 の DNA を 解明 する 。 新春 番組 と し て 定期 的 に 放送 さ れ て おり 、 年末 の 午前 中 に 再 放送 さ れる の が 恒例 と なっ て いる 。',
 'ライブ ドア 社員 で あっ た 初代 代表 取締 役 社長 の 山名 真由 に よっ て 企業 内 起業 の 形 で 創業 。 2005 年 に 株式 会社 ライブ ドア から 分割 さ れ て 設立 。 かつて は ライブ ドア ホールディングス （ 現 ・ LDH ） の 子 会社 で あっ た が 、 ノン コア 事業 の 整理 に ともない 、 株式 会社 ゲオ （ 現 ： 株式 会社 ゲオ ホールディングス ） に 所有 する 全 株式 を 譲渡 し 、 同社 の 完全 子 会社 と なっ た 。 「 ぽす れん 」 「 ゲオ 宅配 レンタル 」 の オン ライン DVD ・ CD ・ コミック レンタル サービス 及び 「 GEO Online 」 と 「 ゲオ アプリ 」 の アプリ ・ ウェブサイト 運営 の 大きく 分け て 2 事業 を 展開 し て いる 。 以前 は DVD 販売 等 の E コマース サービス 「 ぽす れん ストア 」 、 動画 配信 コンテンツ 「 ぽす れん BB 」 や 電子 書籍 配信 サービス の 「 GEO ☆ Books 」 事業 も 行っ て い た 。 オン ライン DVD レンタル 事業 で は 会員 数 は 10 万 人 （ 2005 年 9 月 時点 ） 。 2006 年 5 月 より CD レンタル を 開始 。 同業 他社 に は 、 カルチュア ・ コンビニエンス ・ クラブ が 運営 する 『 TSUTAYA DISCAS 』 の ほか 、 DMM . com が 運営 する 『 DMM . com オン ライン DVD レンタル 』 が ある 。 過去 に は 「 Yahoo ! レンタル DVD 」 と 「 楽天 レンタル 」 の 運営 を 受託 し て い た 。',
 '2005 年 の 一 時期 、 東京 の ラジオ 局 、 InterFM で 、 「 堀江 社長 も 使っ

### 学習

markovifyを使って学習を行う。

In [14]:
model = markovify.Text(data_wakati, state_size=1)

In [15]:
for _ in range(5):
    sentence = model.make_sentence().replace(' ', '')
    print(sentence, end='\n\n')

高から楕円銀河群は山陽本線・小原とのみだりなスプリング・ハイデンについていたの大きな花弁の、ライジング福岡藩の評議員となる。総攻撃に完全になっても、1990年に定年、『プロフェッショナル向けて、彼女は大ファンから、国内オーケストラは広窓を結んだレースウェイブスへの輸送のカロッツェリア・ビデオにて内閣が小腸は57打点と隣接町、歯を獲得にそれぞれ接する。

西安市内の冬季のがエスペランス地区をつなぐアメリカ軍と無線が存在し、一時的に、男性の解明を除いた。

基本的な種

『読売ジャイアンツと、中国仏教反対するために所属した。金はストレートで、名誉な資金をくす玉の中で開催、同じ錯視は捕虜との寿命は自身に始まりで不足につながる一〜ネイビーFCユトレヒト条約を防衛のみの下り降車専用実施。県都モンツァで一つの1次のボールを達成し、反攻するようにマップも存在と同じ3キーパーでは北京首都シュリーランガパトナを記録しながら、『食人とする機能が、ダーバンまでカリフォルニア州選手層のエミリー・ワトソンという。元に多大なる。2015年には同級生およびメゼールとなり鳥栖筑紫野市町教育学区の監督の高い住宅地の清瀧寺村という言葉を利用である。これをあげたが就任後、X線画像、ノースカロライナ大学文学院博士課程修了。デュボスのため、Java系統推定された。ゴールディングのめざましい工業、志願した。イギリス議会は父は河原崎の警戒本部長されることで『ジュノン・コズグリフリッチ・主砲ブラディミール・ゲレーロの舞曲レントラーが進んだが本作を終え、民衆暴動用にもなお車両でプロキックに足場また京滋学生でしか浮かばないとなっている。それ以前から途中でメキシコシティFCウニオン・コミュニケーションズ販売先に結婚しての中のプロダクトプレイスメントにパドレスと、予備予選のある限りで整備などベルイマンの構成を果たした。非常に上昇後、特に前頭10日。2010年6,14歳であり、平原の直前のほうが、ロタール山地はごく少ないらしい。ミジンコダグ・慶長の主張できる。大学出身の改築期間に積雪するにまでレギュラーシーズンあたり210ので負極側扁した。また、宮となる。一部となった。その先に中尉は、32号鴻巣市域直上の選定しての中心に鉄道輸送品が仮定しては街にいかに有用であると、複数の準優勝。サウスウエスト、これに面し、東に増やしつつも

語彙が増えたことで多様な文章が生成されるようになった。